# 🚗⚡ EV2Gym - Démonstration Interactive

Ce notebook présente les fonctionnalités principales d'EV2Gym, une plateforme de simulation pour la recharge intelligente de véhicules électriques avec capacités Vehicle-to-Grid (V2G).

## Sommaire
1. [Installation et Configuration](#installation)
2. [Première Simulation](#premiere-simulation)
3. [Comparaison d'Agents](#comparaison-agents)
4. [Analyse des Résultats](#analyse-resultats)
5. [Configuration Personnalisée](#config-personnalisee)
6. [Visualisations Avancées](#visualisations)

## 1. Installation et Configuration {#installation}

Commençons par importer les modules nécessaires et configurer l'environnement.

In [ ]:
# Imports nécessaires
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

# Ajouter le répertoire parent pour les imports EV2Gym
sys.path.append(str(Path.cwd().parent))

print("✅ Environnement configuré avec succès!")

In [ ]:
# Imports EV2Gym
from ev2gym.models.ev2gym_env import EV2Gym
from ev2gym.rl_agent.reward import SquaredTrackingErrorReward, ProfitMax_TrPenalty_UserIncentives
from ev2gym.rl_agent.state import PublicPST, V2G_profit_max
from ev2gym.baselines.heuristics import RoundRobin, ChargeAsLateAsPossible, ChargeAsFastAsPossible
from tools.demo import SimpleAgents, run_simulation

print("✅ Modules EV2Gym importés avec succès!")

## 2. Première Simulation {#premiere-simulation}

Lançons une première simulation simple pour comprendre le fonctionnement de base.

In [ ]:
# Configuration de base
config_file = "../ev2gym/example_config_files/V2GProfitMax.yaml"

# Créer l'environnement
env = EV2Gym(
    config_file=config_file,
    generate_rnd_game=True,
    state_function=V2G_profit_max,
    reward_function=ProfitMax_TrPenalty_UserIncentives,
    save_replay=False,
    save_plots=False,
    verbose=False
)

print(f"🏗️  Environnement créé:")
print(f"   - Stations de charge: {env.cs}")
print(f"   - Ports totaux: {env.number_of_ports}")
print(f"   - Transformateurs: {env.number_of_transformers}")
print(f"   - Durée simulation: {env.simulation_length} étapes")
print(f"   - V2G activé: {env.config['v2g_enabled']}")

In [ ]:
# Simulation rapide avec l'agent intelligent
print("🚀 Lancement d'une simulation de démonstration...")

env, total_reward, total_cost = run_simulation(
    config_file=config_file,
    agent_type="smart",
    max_steps=50,  # Simulation courte pour la démo
    visualize=False,
    save_results=False
)

print(f"\n📈 Résultats:")
print(f"   - Récompense totale: {total_reward:.2f}")
print(f"   - Coût total: {total_cost:.2f}")
print(f"   - EVs traités: {env.total_evs_spawned}")

## 3. Comparaison d'Agents {#comparaison-agents}

Comparons les performances de différents agents de contrôle.

In [ ]:
# Définir les agents à tester
agents_to_test = {
    'Random': 'random',
    'Charge Rapide': 'fast', 
    'Agent Intelligent': 'smart',
    'Round Robin': 'heuristic'
}

# Stocker les résultats
comparison_results = {}

print("🔬 Comparaison des agents en cours...")

for agent_name, agent_type in agents_to_test.items():
    print(f"   Testing {agent_name}...", end=" ")
    
    try:
        env, reward, cost = run_simulation(
            config_file=config_file,
            agent_type=agent_type,
            max_steps=50,
            visualize=False,
            save_results=False
        )
        
        comparison_results[agent_name] = {
            'reward': reward,
            'cost': cost,
            'evs_served': env.total_evs_spawned,
            'energy_used': env.current_power_usage.sum() if hasattr(env, 'current_power_usage') else 0
        }
        print("✅")
        
    except Exception as e:
        print(f"❌ ({e})")
        comparison_results[agent_name] = {
            'reward': 0, 'cost': float('inf'), 'evs_served': 0, 'energy_used': 0
        }

print("\n📊 Comparaison terminée!")

In [ ]:
# Créer un DataFrame pour l'analyse
df_results = pd.DataFrame(comparison_results).T
print("📋 Résultats de la comparaison:")
print(df_results.round(2))

# Graphiques de comparaison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('🔬 Comparaison des Agents de Contrôle', fontsize=16, fontweight='bold')

# Récompenses
axes[0,0].bar(df_results.index, df_results['reward'], color='skyblue')
axes[0,0].set_title('Récompense Totale')
axes[0,0].set_ylabel('Récompense')
axes[0,0].tick_params(axis='x', rotation=45)

# Coûts
axes[0,1].bar(df_results.index, df_results['cost'], color='lightcoral')
axes[0,1].set_title('Coût Total')
axes[0,1].set_ylabel('Coût')
axes[0,1].tick_params(axis='x', rotation=45)

# EVs servis
axes[1,0].bar(df_results.index, df_results['evs_served'], color='lightgreen')
axes[1,0].set_title('Véhicules Électriques Servis')
axes[1,0].set_ylabel('Nombre d\'EVs')
axes[1,0].tick_params(axis='x', rotation=45)

# Énergie utilisée
axes[1,1].bar(df_results.index, df_results['energy_used'], color='gold')
axes[1,1].set_title('Énergie Totale Utilisée')
axes[1,1].set_ylabel('Énergie (kWh)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Identifier le meilleur agent
best_agent = df_results['reward'].idxmax()
print(f"\n🏆 Meilleur agent: {best_agent} (Récompense: {df_results.loc[best_agent, 'reward']:.2f})")

## 4. Analyse des Résultats {#analyse-resultats}

Analysons en détail les résultats d'une simulation complète.

In [ ]:
# Simulation complète avec sauvegarde des statistiques
print("📊 Lancement d'une simulation complète pour analyse...")

env_analysis, _, _ = run_simulation(
    config_file=config_file,
    agent_type="smart",
    max_steps=None,  # Simulation complète
    visualize=False,
    save_results=True
)

print(f"✅ Simulation terminée après {env_analysis.current_step} étapes")

In [ ]:
# Analyser les statistiques détaillées
if env_analysis.stats:
    stats = env_analysis.stats
    
    print("📈 Statistiques détaillées:")
    print(f"   - Satisfaction utilisateur moyenne: {stats.get('user_satisfaction_mean', 'N/A'):.3f}")
    print(f"   - Énergie totale chargée: {stats.get('total_energy_charged', 'N/A'):.2f} kWh")
    print(f"   - Énergie totale déchargée: {stats.get('total_energy_discharged', 'N/A'):.2f} kWh")
    print(f"   - Profits totaux: {stats.get('total_profits', 'N/A'):.2f} €")
    print(f"   - Violations de contraintes: {stats.get('constraint_violations', 'N/A')}")

# Visualiser l'évolution temporelle
if hasattr(env_analysis, 'current_power_usage'):
    fig, axes = plt.subplots(2, 1, figsize=(15, 8))
    
    # Consommation d'énergie
    time_steps = range(len(env_analysis.current_power_usage))
    axes[0].plot(time_steps, env_analysis.current_power_usage, 'b-', linewidth=2)
    axes[0].set_title('Évolution de la Consommation d\'Énergie')
    axes[0].set_ylabel('Puissance (kW)')
    axes[0].grid(True, alpha=0.3)
    
    # Nombre d'EVs connectés dans le temps
    if hasattr(env_analysis, 'cs_current'):
        total_current = env_analysis.cs_current.sum(axis=0)
        axes[1].plot(time_steps[:len(total_current)], total_current, 'r-', linewidth=2)
        axes[1].set_title('Courant Total des Stations de Charge')
        axes[1].set_ylabel('Courant (A)')
        axes[1].set_xlabel('Étapes de Simulation')
        axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("\n✅ Analyse terminée!")